# LLM 輔助書籍生成程式 - Jupyter Notebook 使用範例
verison: V0.1

本 Notebook 示範如何使用 `bookgen.py` 程式碼 (物件導向版本)，協助您快速生成書籍大綱與章節內容。

**使用步驟：**

1.  **確認檔案:**
    *   請確保您已將更新後的 `bookgen.py` 程式碼檔案和 `config.yaml` 設定檔放在同一個目錄下，或者 Jupyter Notebook 可以訪問到的路徑。
    *   請檢查 `config.yaml` 設定檔，並根據您的需求修改相關參數，**特別是 `gemini_api_key` 請務必替換成您自己的 Gemini API 金鑰。**
    *   如果您要使用檔案上傳功能，請在程式執行目錄下建立名為 `upload_files` 的資料夾 (或者您在 `config.yaml` 中 `upload_directory` 設定的路徑)，並將您的 `.md` 檔案放入該資料夾中。

2.  **執行 Notebook Cell:**
    *   按照 Notebook 中的 Cell 順序，逐個執行。每個 Cell 的 Markdown 說明提供了該 Cell 的功能與操作步驟。
    *   執行過程中，請留意 Cell 的輸出結果與說明，根據需要修改程式碼或設定。

3.  **客製化與擴充:**
    *   您可以根據您的需求，修改 `config.yaml` 設定檔中的參數，例如主題、偏好、提示詞模板等。
    *   如果您熟悉 Python 程式碼，也可以直接修改 `bookgen.py` 檔案，擴充或客製化程式功能。

**開始使用！**

## 步驟 1： 導入程式碼並建立 BookGen 物件

In [ ]:
import bookgen as lbg
import datetime # 導入 datetime 模組

# 建立 BookGen 物件
book_gen = lbg.BookGen()

## 步驟 2： 查看設定檔與程式狀態 (已自動載入)

In [ ]:
# 查看設定檔
print(book_gen.config)

# 查看程式狀態 (目前狀態)
print(book_gen.state)

## 步驟 3： (如果啟用檔案上傳) 上傳檔案並讀取內容

In [ ]:

book_gen.setup_client()
uploaded_file_content = book_gen.upload_files()
book_gen.save_state() # 儲存狀態
#print("\n已讀取檔案內容 (部分顯示):\n", uploaded_file_content[:500] + "..." if uploaded_file_content else "無") # 顯示部分內容

## 步驟 4： 生成書籍大綱

In [ ]:


if not book_gen.book_structure: # 如果 book_structure 為空，才生成大綱，避免重複生成
    book_gen.generate_outline(False)


    book_gen.book_structure, outline_prompt = book_gen.parse_outline()

    book_gen.state['book_structure'] = book_gen.book_structure # 更新狀態
    book_gen.state['prompt_log'] = book_gen.prompt_log + [{'type': 'outline_prompt', 'prompt': outline_prompt, 'timestamp': datetime.datetime.now().isoformat(), 'uploaded_files': book_gen.uploaded_files_list}] # 記錄 prompt
    book_gen.save_state() # 儲存狀態
else:
    print("已載入書籍大綱，跳過大綱生成。")

#book_gen.book_structure # 顯示書籍結構

## 步驟 5： 匯出章節摘要並上傳

In [ ]:
export_level = 'chapters_selected' # 可選: 'all', 'outline_only', 'chapters_selected'
book_gen.export_book_to_markdown(export_level=export_level)
uploaded_file_content = book_gen.upload_files()
book_gen.save_state() # 儲存狀態

## 步驟 6： 選擇章節/節點並生成詳細內容

In [ ]:
chapter_section_titles = book_gen.get_chapter_section_titles()
# get index list of chapter_section_titles
index_list = []
for chapter_section_title in chapter_section_titles:
    if chapter_section_title['type']=='section':
        index_list.append(chapter_section_title['index'])
print(index_list)
chapter_section_titles

In [ ]:
import datetime # 導入 datetime 模組
chapter_section_titles = book_gen.get_chapter_section_titles()
selected_titles = ['4.1', '4.2', '4.3'] #  <---  請修改為您想要生成詳細內容的章節/節點 index

for item in chapter_section_titles:
    if str(item['index']) in selected_titles: # 將 index 轉為字串比對
        if item['type'] == 'section': # 節點
            chapter_title = item['chapter_title']
            section_title = item['title']
            if not book_gen.book_structure[chapter_title]['sections'][section_title]['content']: # 如果節點內容為空，才生成
                file_path = book_gen.generate_section_content(item['index'],chapter_title, section_title)
                prompt, content_text= book_gen.parse_section(file_path)
                book_gen.book_structure[chapter_title]['sections'][section_title]['content'] = content_text # 更新節點內容
                book_gen.state['book_structure'] = book_gen.book_structure # 更新狀態
                book_gen.state['prompt_log'] = book_gen.prompt_log + [{'type': 'chapter_content_prompt', 'chapter': chapter_title, 'section': section_title, 'prompt': prompt, 'timestamp': datetime.datetime.now().isoformat(), 'uploaded_files': book_gen.uploaded_files_list}] # 記錄 prompt
                book_gen.save_state() # 儲存狀態
                print(f"\n已生成 {chapter_title} - {section_title} 的詳細內容。")
            else:
                print(f"已載入 {chapter_title} - {section_title} 的詳細內容，跳過生成。")
        elif item['type'] == 'chapter': # 章節 (如果您想要章節也生成詳細內容，可以加入此判斷)
            pass # 章節層級目前只生成摘要，詳細內容針對節點

#book_gen.book_structure # 再次顯示更新後的書籍結構，包含章節內容

## 步驟 7：完成匯出書籍與狀態

In [ ]:
export_level = 'all' # 可選: 'all', 'outline_only', 'chapters_selected'
book_gen.export_book_to_markdown(export_level=export_level)
#uploaded_file_content = book_gen.upload_files()
book_gen.save_state() # 儲存狀態

## 步驟 8： 查看 Prompt 紀錄 (可選)

In [ ]:
book_gen.state['prompt_log'] # 顯示 Prompt 紀錄

您可以根據您的需求，重複執行步驟 6，生成更多章節內容，並匯出更新後的書籍。



# Debug

In [ ]:
book_gen.state